# Dropout(드롭아웃) 설명
- 과적합의 이해 - 학습한 결과가 학습 데이터에는 매우 잘 맞지만, 학습 데이터에만 너무 꼭 맞춰져 있어, 그
외의 데이터에는 잘 맞지 않음.
- 학습시 전체 신경망 중 일부만을 사용하도록 하는 것.
- 즉, 학습 단계마다 일부 뉴런을 제거(사용하지 않도록)함으로써, 일부 특징이 특정 뉴런에 고정되는 것을 막
아 가중치의 균형을 잡도록 한다.
- 학습 시 일부 뉴런을 학습시키지 않기 때문에 신경망이 충분히 학습되기까지의 시간은 조금 더 오래 걸리는
편이다

In [0]:
import os,warnings
warnings.filterwarnings(action='ignore')

In [0]:
from IPython.display import display, Image

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [0]:
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)
type(mnist)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [0]:
# 데이터의 개수
print(mnist.test.num_examples, mnist.train.num_examples, mnist.validation.num_examples)
# 데이터의 행열 사이즈
print(mnist.train.labels.shape, mnist.train.images.shape)
print(mnist.test.labels.shape, mnist.test.images.shape)
print(mnist.validation.labels.shape, mnist.validation.images.shape)

10000 55000 5000
(55000, 10) (55000, 784)
(10000, 10) (10000, 784)
(5000, 10) (5000, 784)


In [0]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

In [0]:
keep_prob = tf.placeholder(tf.float32)
W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X, W1))
L1 = tf.nn.dropout(L1, keep_prob) # dropout
W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1, W2))
L2 = tf.nn.dropout(L2, keep_prob) # 뒤의 뉴런을 0.8만 이용 (학습시 해당 계층의 80%만 이용)
W3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L2, W3)
print(W3)
print(model)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
<tf.Variable 'Variable_2:0' shape=(256, 10) dtype=float32_ref>
Tensor("MatMul_2:0", shape=(?, 10), dtype=float32)


In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [0]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [0]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
print(total_batch)

550


In [0]:
for epoch in range(15):
    total_cost = 0
    for i in range(total_batch):
        # 배치 사이즈만큼 데이터 가져오기
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # 입력값 : batch_xs, 출력값 : batch_ys
        # 최적화를 수행 후, 손실을 구한다.
        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X:batch_xs, Y:batch_ys,
                                          keep_prob:0.8})
        # 총 손실 계산
        total_cost = total_cost + cost_val
    print("Epoch : %4d" %(epoch + 1),
          '평균 Cost = ', "{:.3f}".format(total_cost/total_batch))
print("최적화 완료!")

Epoch :    1 평균 Cost =  0.433
Epoch :    2 평균 Cost =  0.170
Epoch :    3 평균 Cost =  0.116
Epoch :    4 평균 Cost =  0.088
Epoch :    5 평균 Cost =  0.074
Epoch :    6 평균 Cost =  0.060
Epoch :    7 평균 Cost =  0.054
Epoch :    8 평균 Cost =  0.048
Epoch :    9 평균 Cost =  0.044
Epoch :   10 평균 Cost =  0.037
Epoch :   11 평균 Cost =  0.034
Epoch :   12 평균 Cost =  0.031
Epoch :   13 평균 Cost =  0.028
Epoch :   14 평균 Cost =  0.026
Epoch :   15 평균 Cost =  0.026
최적화 완료!
